<a href="https://colab.research.google.com/github/Archelephant/Archelephant/blob/main/X5_Hallucinations_Train_LightAML_only_answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Важно! Не забудь включить среду исполнения "GPU" перед стартом ноутбука!

In [ ]:
#установка LightAutoML версии, совместимой с Python 3.10 и Colab
!pip install -U lightautoml==0.3.8b1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.3/382.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 35.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 108.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 103.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manyli

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
import os
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, log_loss
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import torch

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.text_presets import TabularNLPAutoML
from lightautoml.tasks import Task

from lightautoml.addons.interpretation import LimeTextExplainer, L2XTextExplainer

import transformers
transformers.logging.set_verbosity(50)

import pickle

In [ ]:
#Присоединяем свой диск
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Задаем путь к папке, где лежит датасет
startpath = '/content/drive/MyDrive/X5AIHack'

In [ ]:
#Проверяем, что датасет на месте
os.chdir(startpath)
os.listdir()

['train.csv']

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

,line_id,summary,question,answer,is_hallucination
0,0,Херманус Питер (Дик) Логгере (нидерл. Hermanus...,В каком городе проходил чемпионат мира по хокк...,В Хилверсюме.,1
1,1,Ходуткинские горячие источники (Худутские горя...,Как называется район в который входят источники?,Елизовским районом,0
2,2,Чёрная вдова (лат. Latrodectus mactans) — вид ...,Для кого опасны пауки-бокоходы?,Для рыб.,1
3,3,"Рысь — река в России, протекает по территориям...",Какова длина реки Рысь?,5 км.,1
4,4,"И́се (яп. 伊勢市), ранее Удзиямада — город в Япон...",Что такое Исе?,"Исе — это небольшой город в Японии, который не...",1


In [ ]:
df.columns

Index(['line_id', 'summary', 'question', 'answer', 'is_hallucination'], dtype='object')

In [ ]:
df.shape

(1050, 5)

In [ ]:
#Базовые настройки для дообучения классификатора LightAutoML на базе BERT-модели, которую будем брать с Hugging-Face
np.random.seed(42)

train, test = train_test_split(df, test_size=0.2, random_state=42)

roles = {
    'text': ['answer'],
    'drop': ['line_id', 'summary', 'question'], #в данном примере для классификатора использовал только колонку 'answer'
    'target': 'is_hallucination'  #надо бы придумать, как использовать еще и 'summary' и 'question'
}

task = Task('binary')

automl = TabularNLPAutoML(
    task=task,
    timeout=3600,
    cpu_limit=1,
    gpu_ids='0',
    general_params={
        'nested_cv': False,
        'use_algos': [['nn']]
    },
    autonlp_params={
        'sent_scaler': 'l2'
    },
    text_params={
        'lang': 'ru',
        'bert_model': 'DeepPavlov/rubert-base-cased'
    },
    nn_params={
        'opt_params': {'lr': 1e-5},
        'max_length': 128,
        'bs': 32,
        'n_epochs': 7,
    }

)

In [ ]:
%%time
oof_pred = automl.fit_predict(train, roles=roles, verbose = 10)
test_pred = automl.predict(test)
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)

print('Check scores:')
print('OOF score: {}'.format(roc_auc_score(train[roles['target']].values[not_nan], oof_pred.data[not_nan][:, 0])))
print('TEST score: {}'.format(roc_auc_score(test[roles['target']].values, test_pred.data[:, 0])))

[08:41:07] Stdout logging level is DEBUG.


INFO:lightautoml.automl.presets.base:Stdout logging level is DEBUG.


[08:41:07] Model language mode: ru


INFO3:lightautoml.automl.presets.text_presets:Model language mode: ru


[08:41:07] Task: binary



...

[08:47:20] Automl preset training completed in 372.86 seconds



INFO:lightautoml.automl.presets.base:Automl preset training completed in 372.86 seconds



[08:47:20] Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (3 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN__linear_layer_0) 



INFO:lightautoml.automl.presets.base:Model description:
Final prediction for new objects (level 0) = 
	 1.00000 * (3 averaged models Lvl_0_Pipe_0_Mod_0_TorchNN__linear_layer_0) 

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose par

Check scores:
OOF score: 0.9696260754654916
TEST score: 0.9438344977769714
CPU times: user 5min 32s, sys: 25.3 s, total: 5min 57s
Wall time: 6min 25s


In [ ]:
#Посчитаем привычную нам F1-меру вместо ROC AUC
print('F1 Test Score: {}'.format(f1_score(test[roles['target']].values, test_pred.data.round())))

F1 Test Score: 0.8857142857142858
